In [1]:
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing as preproc
from scipy import stats
import numpy as np

In [2]:
df=pd.read_parquet("diabetic_data_cleaned_pca_som_ae.parquet")

In [3]:
df.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,...,ae_3,ae_4,ae_5,ae_6,ae_7,ae_8,ae_9,ae_10,ae_11,ae_12
0,Caucasian,Female,[10-20),1,1,7,3,59,0,18,...,25.693628,11.905568,30.468431,22.546497,22.514284,26.188181,13.847610,4.928596,21.504631,13.883195
1,AfricanAmerican,Female,[20-30),1,1,7,2,11,5,13,...,30.512671,28.937412,56.934689,21.478495,23.168261,20.561810,8.013588,16.540028,43.731636,31.864979
2,Caucasian,Male,[30-40),1,1,7,2,44,1,16,...,32.077106,22.075180,30.750517,35.346027,30.831038,34.111694,19.892376,10.097866,20.811745,18.822224
3,Caucasian,Male,[40-50),1,1,7,1,51,0,8,...,58.738827,30.175955,36.444820,27.964182,35.391453,37.752415,29.441738,12.080683,50.726456,33.423729
4,Caucasian,Male,[50-60),2,1,2,3,31,6,16,...,49.605930,17.417633,43.298721,33.306580,35.973259,47.085152,82.284737,25.919033,14.402472,38.670834


In [4]:
df.columns, df.shape

(Index(['race', 'gender', 'age', 'admission_type_id',
        'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
        'num_lab_procedures', 'num_procedures', 'num_medications',
        'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
        'diag_2', 'diag_3', 'number_diagnoses', 'metformin', 'repaglinide',
        'glimepiride', 'glipizide', 'glyburide', 'pioglitazone',
        'rosiglitazone', 'insulin', 'glyburide-metformin', 'change',
        'diabetesMed', 'readmitted', 'PC1', 'PC2', 'PC3', 'som_jaccard_0',
        'som_jaccard_1', 'som_euc_0', 'som_euc_1', 'ae_0', 'ae_1', 'ae_2',
        'ae_3', 'ae_4', 'ae_5', 'ae_6', 'ae_7', 'ae_8', 'ae_9', 'ae_10',
        'ae_11', 'ae_12'],
       dtype='object'),
 (100244, 49))

In [5]:
DB_COLUMNS = [
  'PC1','PC2', 'PC3','ae_0', 'ae_1', 'ae_2', 'ae_3', 'ae_4',
  'ae_5', 'ae_6', 'ae_7', 'ae_8', 'ae_9', 'ae_10', 'ae_11',
  'ae_12',
]

df[DB_COLUMNS]

,PC1,PC2,PC3,ae_0,ae_1,ae_2,ae_3,ae_4,ae_5,ae_6,ae_7,ae_8,ae_9,ae_10,ae_11,ae_12
0,-0.214843,0.070744,-0.194516,8.275141,12.456257,13.877572,25.693628,11.905568,30.468431,22.546497,22.514284,26.188181,13.847610,4.928596,21.504631,13.883195
1,0.417941,-0.522845,0.160733,6.040537,29.757124,33.032806,30.512671,28.937412,56.934689,21.478495,23.168261,20.561810,8.013588,16.540028,43.731636,31.864979
2,-0.129162,-0.131116,-0.050765,25.172379,3.856527,12.829103,32.077106,22.075180,30.750517,35.346027,30.831038,34.111694,19.892376,10.097866,20.811745,18.822224
3,-0.333245,-0.143083,-0.009183,29.618387,21.520760,45.986839,58.738827,30.175955,36.444820,27.964182,35.391453,37.752415,29.441738,12.080683,50.726456,33.423729
4,0.640729,-0.446888,-0.059434,40.608124,65.410851,31.280500,49.605930,17.417633,43.298721,33.306580,35.973259,47.085152,82.284737,25.919033,14.402472,38.670834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100239,-0.226515,0.049629,-0.151316,30.157810,18.086729,24.437574,20.127266,23.402134,7.712886,13.635165,44.357796,30.622820,7.171422,14.741157,39.390068,23.661821
100240,0.266458,-0.086568,-0.008378,4.069397,22.508503,1.567279,20.514818,8.234191,18.906248,18.680935,17.009922,15.069460,14.574155,8.597450,26.465485,7.286322
100241,-0.277035,-0.027778,-0.406056,26.163000,9.188273,34.756756,26.739008,22.499334,13.459891,11.029498,37.792614,23.588331,10.348755,5.263132,21.847214,22.683990
100242,0.291241,0.333055,0.077571,15.308062,38.008709,31.012506,24.283443,21.043257,22.479326,18.605734,16.524364,24.074730,7.971524,15.340425,39.851978,10.798032


In [6]:
X=df[DB_COLUMNS]

# Scale the independent variables
#scaler = StandardScaler()
X_scaled = preproc.MinMaxScaler().fit_transform(X)

In [7]:
# Apply DBSCAN
epsilon = 0.37 # You may need to adjust this parameter
min_samples = 10 # You may need to adjust this parameter
dbscan = DBSCAN(eps=epsilon, min_samples=min_samples, n_jobs=-1)
#dbscan.fit(X_scaled)
dbscan.fit(X_scaled)

DBSCAN(eps=0.37, min_samples=10, n_jobs=-1)

In [8]:
# Extract outlier labels
outlier_mask = dbscan.labels_ == -1

# Get the outliers
outliers = X[outlier_mask]

print(len(outliers))

1124


In [9]:
len(set(dbscan.labels_))

2

In [10]:
# Analyze outliers
print(len(df[outlier_mask]))

1124


In [11]:
non_outlier_mask=dbscan.labels_ != -1

In [12]:
print(len(df[non_outlier_mask]))

99120


In [13]:
def display_df(df: pd.DataFrame, title: str = ""):
  display(df.style.hide(axis="index").set_caption(title))

In [14]:
TGT = "readmitted"

In [15]:
prop_total = df.loc[:, TGT].value_counts().reset_index()
display_df(prop_total.sort_values("readmitted", ascending = False), "full dataset")

prop_outs = df.loc[list(outlier_mask), TGT].value_counts().reset_index()
display_df(prop_outs.sort_values("readmitted", ascending = False), "DBscan outliers")

stats.chi2_contingency(np.array([prop_total["count"].values, prop_outs["count"].values]))

readmitted,count
NO,53821
>30,35173
<30,11250


readmitted,count
NO,584
>30,415
<30,125


Chi2ContingencyResult(statistic=1.6973040404236048, pvalue=0.42799146712975367, dof=2, expected_freq=array([[53801.74039145, 35193.38915634, 11248.87045221],
       [  603.25960855,   394.61084366,   126.12954779]]))

In [16]:
output_df=df[non_outlier_mask]

In [17]:
output_df.to_csv("diabetic_data_cleaned_pca_som_ae_dbscan.csv")
output_df.to_parquet("diabetic_data_cleaned_pca_som_ae_dbscan.parquet")